In [110]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing,svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


#ignore warning 
import warnings
warnings.filterwarnings('ignore')

In [111]:
df = quandl.get("WIKI/GOOGL")
df

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-21,1092.57,1108.70,1087.21,1094.000,1990515.0,0.0,1.0,1092.570000,1108.700000,1087.210000,1094.000000,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.150,3418154.0,0.0,1.0,1080.010000,1083.920000,1049.640000,1053.150000,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.550,2413517.0,0.0,1.0,1051.370000,1066.780000,1024.870000,1026.550000,2413517.0


In [112]:
#High and Low tells us about the volatility for the day.
#Open and Close us the if the price went up or down, by what degree

df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume',]]
df

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0
...,...,...,...,...,...
2018-03-21,1092.570000,1108.700000,1087.210000,1094.000000,1990515.0
2018-03-22,1080.010000,1083.920000,1049.640000,1053.150000,3418154.0
2018-03-23,1051.370000,1066.780000,1024.870000,1026.550000,2413517.0


In [113]:
df['HL_PCT'] = (df['Adj. High']- df['Adj. Close']) / df['Adj. Close'] * 100.0
df

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT
Date,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,3.712563
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,0.710922
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,3.729433
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,6.417469
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,1.886792
...,...,...,...,...,...,...
2018-03-21,1092.570000,1108.700000,1087.210000,1094.000000,1990515.0,1.343693
2018-03-22,1080.010000,1083.920000,1049.640000,1053.150000,3418154.0,2.921711
2018-03-23,1051.370000,1066.780000,1024.870000,1026.550000,2413517.0,3.918952


In [114]:
df['PCT_change'] = (df['Adj. Close']- df['Adj. Open']) /df['Adj. Open'] * 100.0
df

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,3.712563,0.324968
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,0.710922,7.227007
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,3.729433,-1.227880
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,6.417469,-5.726357
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,1.886792,1.183658
...,...,...,...,...,...,...,...
2018-03-21,1092.570000,1108.700000,1087.210000,1094.000000,1990515.0,1.343693,0.130884
2018-03-22,1080.010000,1083.920000,1049.640000,1053.150000,3418154.0,2.921711,-2.487014
2018-03-23,1051.370000,1066.780000,1024.870000,1026.550000,2413517.0,3.918952,-2.360729


In [115]:
df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
df

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0
...,...,...,...,...
2018-03-21,1094.000000,1.343693,0.130884,1990515.0
2018-03-22,1053.150000,2.921711,-2.487014,3418154.0
2018-03-23,1026.550000,3.918952,-2.360729,2413517.0


In [116]:
# Replacing na with -9999

forecast_col = 'Adj. Close'

In [117]:
df.fillna('-9999, inplace =True')

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0
...,...,...,...,...
2018-03-21,1094.000000,1.343693,0.130884,1990515.0
2018-03-22,1053.150000,2.921711,-2.487014,3418154.0
2018-03-23,1026.550000,3.918952,-2.360729,2413517.0


In [118]:
#It will take anything and take to coiling--> round up
forecast_out = int(math.ceil(0.01 *len(df)))

In [119]:
#Creating a label

df['label'] =df[forecast_col].shift(-forecast_out)
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.078238
2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.839414
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,1.886792,1.183658,9188600.0,71.219849


In [120]:
df.dropna(inplace= True)
df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2018-01-30,1177.37,0.896914,-0.029718,1792602.0,1094.00
2018-01-31,1182.22,0.346805,-0.134312,1643877.0,1053.15
2018-02-01,1181.59,0.495942,0.476195,2774967.0,1026.55
2018-02-02,1119.20,1.081129,-0.729098,5798880.0,1054.09
2018-02-05,1068.76,4.325574,-2.893850,3742469.0,1006.94


In [121]:
X = np.array(df.drop(['label'],1))
X

array([[ 5.03228418e+01,  3.71256291e+00,  3.24967503e-01,
         4.46590000e+07],
       [ 5.43226889e+01,  7.10922353e-01,  7.22700723e+00,
         2.28343000e+07],
       [ 5.48693765e+01,  3.72943327e+00, -1.22788010e+00,
         1.82561000e+07],
       ...,
       [ 1.18159000e+03,  4.95941909e-01,  4.76194525e-01,
         2.77496700e+06],
       [ 1.11920000e+03,  1.08112938e+00, -7.29098295e-01,
         5.79888000e+06],
       [ 1.06876000e+03,  4.32557356e+00, -2.89384977e+00,
         3.74246900e+06]])

In [123]:
y = np.array(df['label'])
y

array([  69.0782379 ,   67.83941377,   68.91272699, ..., 1026.55      ,
       1054.09      , 1006.94      ])

In [125]:
X = preprocessing.scale(X)
#X = X[:-forecast_out+1]


In [126]:
df.dropna(inplace=True)
y = np.array(df['label'])

len(X),len(y)

(3389, 3389)

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [131]:
clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test,y_test)

In [132]:
accuracy

0.9788024520087277

In [134]:
print(forecast_out)

35


In [143]:
# #changing the algothrim
# clf = svm.SVR(kernel= 'linear')
# clf.fit(X_train, y_train)
# accuracy = clf.score(X_test,y_test)

In [144]:
accuracy

0.9784315454829778